In [6]:

import os
import wget
import ssl

data_dir = 'data'
url = 'https://s3.ap-northeast-2.amazonaws.com/data10902/petition/petition_sampled.csv'
ssl._create_default_https_context = ssl._create_unverified_context

if not os.path.exists(data_dir):
    os.mkdir(data_dir)

wget.download(url, data_dir)

'data/petition_sampled (1).csv'

In [7]:
import pandas as pd
df = pd.read_csv("./data/petition_sampled.csv")
df_sorted= df.sort_values(by='start')

df_sorted['time'] = df_sorted['start'].map(lambda x: x[:7])

time_slice = list(df_sorted['time'].value_counts().sort_index())
print(time_slice,sum(time_slice))

[83, 906, 298, 1446, 926, 1514, 1304, 1097, 1217, 1301, 1246, 1291, 1390, 1155, 1277, 1187, 439] 18077


In [8]:
time_tag = sorted(list(set(df_sorted['time'])))
time_tag

['2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12']

In [9]:
from gensim.corpora.dictionary import Dictionary
from konlpy.tag import Okt
twit = Okt()

def tokenizer(doc):
    return [token for token in twit.nouns(doc) if len(token) > 1] 
texts = [tokenizer(news) for news in df_sorted['content']]

dictionary = Dictionary(texts)
print('#number of initial unique words in documents:', len(dictionary))

dictionary.filter_extremes(keep_n=2000, no_below=5, no_above=0.5)
print('#number of unique words after removing rare and common words:', len(dictionary))

copus = [dictionary.doc2bow(text) for text in texts]
print(' #number of unique tokens: %d' %len(dictionary))


In [ ]:
#from gensim.models import CoherenceModel

#cm = CoherenceModel(model=model, corpus=copus, coherence='u_mass')
#coherence = cm.get_coherence()
#print(coherence)
#show_coherence(corpus,dictionary,start=6,end=15)

In [ ]:
from gensim.models import LdaSeqModel
import warnings
warnings.filterwarnings("ignore")

ldaseq = LdaSeqModel(corpus=copus,id2word=dictionary,time_slice=time_slice,
                     num_topics=6,random_state=7)

KeyboardInterrupt: 

In [ ]:
def get_topic_words(topic,top_n_words=10):
    words,_ = zip(*topic)
    return words[:top_n_words]

def print_topic_times(model, topic, time_tag, top_n_words=10):
    topic_times = model.print_topic_times(topic)
    for i in range(len(topic_times)):
        print(time_tag[i]+':',get_topic_words(topic_times[i],top_n_words))

In [ ]:
print_topic_times(ldaseq,4,time_tag,8)

In [ ]:
import numpy as np
def get_topic_trends(model,corpus,time_slice):
    dtm = model.dtm_vis(0,corpus)
    result = []
    start = 0
    for index in time_slice:
        result.append(np.array(dtm[0][start:start+index]).mean(axis=0))
        start += index
    return np.array(result)
topic_trends = get_topic_trends(ldaseq,copus,time_slice)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

topic_titles = ['Topic '+str(i) for i in range(6)]
fig, axes = plt.subplots(2,3,sharex='col',figsize=(12,6))
for i, (title,ax) in enumerate(zip(topic_titles,axes.ravel())):
    ax.set_title(title)
    ax.set_xticks([0,5,10,15])
    ax.set_xticklabels(time_tag[::5])
    ax.plot(topic_trends[:,i])
plt.savefig('dtm.png')
plt.show()

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

doc_topic, topic_term, doc.lengths, term_frequency, vocab = ldaseq.dtm_vis(time=0,
                                                                           corpus=copus)
vis_dtm = pyLDAvis.prepare(topic_term_dists=topic_term, doc_topic_dists=doc_topic,doc_lengths=doc_lengths
                           vocab=vocab,term_frequency=frequency)
pyLDAvis.display(vis_dtm)
